In [1]:
# import cv2
#
# img = cv2.imread("datasets/ios/training_set/0A967511-9D13-40D3-9A95-20125CDB25CD.png")
# img.shape
#

In [2]:
# img_resized = cv2.resize(img, (256, 256))
# img_resized.shape
# cv2.imshow("Original", img)
# cv2.imshow("Resized image", img_resized)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
from keras.utils.vis_utils import plot_model
from keras.layers import Input, Dense, Dropout, \
                         RepeatVector, LSTM, concatenate, \
                         Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras import *
from model.classes.model.Config import *
from model.classes.model.AModel import *

Using TensorFlow backend.


In [5]:
input_shape = (256, 256, 3)
output_size = 19
output_path = '../bin'
image_model = Sequential()
image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', input_shape=input_shape))
image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu'))
image_model.add(MaxPooling2D(pool_size=(2, 2)))
image_model.add(Dropout(0.25))

image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
image_model.add(MaxPooling2D(pool_size=(2, 2)))
image_model.add(Dropout(0.25))

image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
image_model.add(MaxPooling2D(pool_size=(2, 2)))
image_model.add(Dropout(0.25))

image_model.add(Flatten())
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))

image_model.add(RepeatVector(CONTEXT_LENGTH))

visual_input = Input(shape=input_shape)
encoded_image = image_model(visual_input)

language_model = Sequential()
language_model.add(LSTM(128, return_sequences=True, input_shape=(CONTEXT_LENGTH, output_size)))
language_model.add(LSTM(128, return_sequences=True))

textual_input = Input(shape=(CONTEXT_LENGTH, output_size))
encoded_text = language_model(textual_input)

decoder = concatenate([encoded_image, encoded_text])

decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder = Dense(output_size, activation='softmax')(decoder)

model = Model(inputs=[visual_input, textual_input], outputs=decoder)

optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

plot_model(model, to_file='./pix2code.png', show_shapes=True, show_layer_names=True)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 48, 19)       0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 48, 1024)     104098080   input_3[0][0]                    
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 48, 128)      207360      input_4[0][0]                    
____________________________________________________________________________________________